# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [ ]:
# Load the dataset
import pandas as pd
df = pd.read_csv('OnlineRetail.csv',encoding = 'ISO-8859-1' )
# Preprocess as per the instructions above | We have already done in TASK 2
df = df.dropna()
df = df[(df.Quantity > 0) & (df.Country == 'United Kingdom')]
df['Description'] = df['Description'].str.strip().str.lower()
print(f"Filtered rows: {len(df)}")
print(f"Unique items: {df['Description'].nunique()}")
# Your Code Here
# for Basket
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket = basket.applymap(lambda x: 1 if x > 0 else 0)
transactions = basket.apply(lambda row: set(row[row > 0].index), axis=1).tolist()
print(f"Total transactions: {len(transactions)}")
print(f"Sample transaction: {transactions[0] if transactions else 'None'}")

Filtered rows: 354345
Unique items: 3833


C:\Users\nikunj\AppData\Local\Temp\ipykernel_3584\3736625478.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [ ]:
# Implement apriori functions below

def get_frequent_itemsets(transactions, min_support):
    frequent_itemsets = {}
    item_counts = {}

    for transaction in transactions:
        for item in transaction:
            item_counts[frozenset([item])] = item_counts.get(frozenset([item]), 0) + 1

    num_transactions = len(transactions)
    L1 = {itemset: count / num_transactions for itemset, count in item_counts.items() if (count / num_transactions) >= min_support}
    frequent_itemsets.update(L1)

    Lk_minus_1 = L1
    k = 2

    while Lk_minus_1:
        Ck = generate_candidates(Lk_minus_1.keys(), k)
        support_counts = calculate_support(transactions, Ck)
        Lk = {itemset: count / num_transactions for itemset, count in support_counts.items() if (count / num_transactions) >= min_support}

        if Lk:
            frequent_itemsets.update(Lk)
            Lk_minus_1 = Lk
            k += 1
        else:
            break

    return frequent_itemsets

def generate_candidates(prev_frequent_itemsets, k):
    candidates = set()
    list_prev_itemsets = list(prev_frequent_itemsets)

    for i in range(len(list_prev_itemsets)):
        for j in range(i + 1, len(list_prev_itemsets)):
            itemset1 = list_prev_itemsets[i]
            itemset2 = list_prev_itemsets[j]

            if k > 2:
                sorted_itemset1 = sorted(list(itemset1))
                sorted_itemset2 = sorted(list(itemset2))
                if sorted_itemset1[:-1] == sorted_itemset2[:-1]:
                    new_itemset = frozenset(itemset1.union(itemset2))
                    is_valid = all(frozenset(subset) in prev_frequent_itemsets for subset in combinations(new_itemset, k - 1))
                    if is_valid:
                        candidates.add(new_itemset)
            else:
                new_itemset = itemset1.union(itemset2)z
                if len(new_itemset) == k:
                    candidates.add(new_itemset)


    return candidates

def calculate_support(transactions, candidates):
    support_counts = {candidate: 0 for candidate in candidates}
    for transaction in transactions:
        for candidate in candidates:
            if candidate.issubset(transaction):
                support_counts[candidate] += 1
    return support_counts    

## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [ ]:
# Function to generate rules from frequent itemsets

def generate_rules(frequent_itemsets, min_confidence):
    rules = []

    for itemset, support_itemset in frequent_itemsets.items():
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset - antecedent

                    support_antecedent = frequent_itemsets.get(antecedent, 0)
                    if support_antecedent > 0:
                        confidence = support_itemset / support_antecedent
                        if confidence >= min_confidence:
                            support_consequent = frequent_itemsets.get(consequent, 0)
                            lift = confidence / support_consequent if support_consequent > 0 else float('inf')

                            rules.append({
                                'antecedents': antecedent,
                                'consequents': consequent,
                                'support': support_itemset,
                                'confidence': confidence,
                                'lift': lift
                            })

    return rules

    

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [ ]:

# Step 5: Run the algorithm
min_support = 0.01 
min_confidence = 0.5 

frequent_itemsets = get_frequent_itemsets(transactions, min_support)
print(f"Frequent itemsets found: {len(frequent_itemsets)}")

rules = generate_rules(frequent_itemsets, min_confidence)
print(f"Association rules generated: {len(rules)}")

# Step 6: Output results
print("\n## Top 10 Frequent Itemsets (by support)")
sorted_frequent_itemsets = sorted(frequent_itemsets.items(), key=lambda item: item[1], reverse=True)
for itemset, support in sorted_frequent_itemsets[:10]:
    print(f"Itemset: {list(itemset)}, Support: {support:.4f}")

print("\n" + "-"*50 + "\n")

print("## Top 10 Association Rules (by lift)")
sorted_rules = sorted(rules, key=lambda rule: rule['lift'], reverse=True)
for rule in sorted_rules[:10]:
    print(f"Rule: {list(rule['antecedents'])} => {list(rule['consequents'])}")
    print(f"  Support: {rule['support']:.4f}, Confidence: {rule['confidence']:.4f}, Lift: {rule['lift']:.4f}")
    print("-" * 20)
